In [8]:
# import torch
# from transformers import *

# model_class, tokenizer_class, pretrained_weights  = (BertModel, BertTokenizer, 'bert-base-uncased')
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights).cuda()
# model.train()

In [2]:
from bert_model import *
my_model = mymodel().cuda()
my_model.train()
print('ok')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

ok


In [9]:
# Parameters:
lr = 1e-3
max_grad_norm = 1.0
num_training_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(my_model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

In [2]:
f_label = open('main_action.txt')
labels = f_label.readlines()
gts = [x.strip() for x in labels]

In [1]:
f = open('corpus_100000.txt')
texts = f.readlines()

In [78]:
# for i in range(len(texts)):
for i in range(1):
    label = texts[i].split('\t')[0]    
    sen = texts[i].split('\t')[1].strip()
    
    label_idx = gts.index(label)
    torch_label = torch.from_numpy(np.asarray([label_idx])).cuda() 
    
    bert_out = my_model.bert_layer(sen) # (batch, seq_len, emb_dim)    
#     model_out = model(sen_idx)[0] 
    fc_out = my_model.fc_layer(bert_out)
    cls_out = fc_out[:,0:1,:].squeeze(1)
    
    loss = my_model.cls_loss(torch_label, cls_out)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(my_model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

In [79]:
for name, param in my_model.named_parameters():
    if name == 'model.embeddings.word_embeddings.weight':
        print(name, param.data)
    if name == 'model.encoder.layer.1.attention.self.query.weight':
        print(name, param.data)
    if name == 'matrix.weight':
        print(name, param.data)
    if name == 'matrix.bias':
        print(name, param.data)        
        

        
#     if param.requires_grad:
#         print(name, param.data)

model.embeddings.word_embeddings.weight tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       device='cuda:0')
model.encoder.layer.1.attention.self.query.weight tensor([[ 0.0303, -0.0246,  0.0064,  ..., -0.0333, -0.0601,  0.0664],
        [-0.0123, -0.0221, -0.0302,  ..., -0.0324,  0.0519,  0.0496],
        [-0.0740, -0.0510, -0.0088,  ...,  0.0180,  0.0118, -0.0496],
        ...,
        [ 0.0563, -0.0185, -0.0335,  ...,  0.0111,  0.0094, -0.0260],
        [ 0.0086, -0.0139, -0.0069,  ..., -0.0046,  0.0224, -0.0608],
        [-0.0080, -0.0141,  0.0540,  ...,  0.1033, -0.0529,  0.0210]],
       device='cud

In [67]:
list(my_model.parameters())[5]

Parameter containing:
tensor([[-0.0157,  0.0245, -0.0258,  ...,  0.0133,  0.0772,  0.0566],
        [-0.0315,  0.0362, -0.0417,  ..., -0.0518,  0.1390,  0.0078],
        [ 0.0118,  0.0329,  0.0117,  ..., -0.0295,  0.0256, -0.0448],
        ...,
        [-0.0087,  0.0491,  0.0566,  ...,  0.0256,  0.0526, -0.0541],
        [-0.0213,  0.0922,  0.0627,  ..., -0.1049,  0.0589,  0.0448],
        [ 0.0026, -0.0948,  0.0117,  ..., -0.0199, -0.0503, -0.0096]],
       device='cuda:0', requires_grad=True)

In [47]:
for name, param in my_model.named_parameters():
    if param.requires_grad:
        print(name)

model.embeddings.word_embeddings.weight
model.embeddings.position_embeddings.weight
model.embeddings.token_type_embeddings.weight
model.embeddings.LayerNorm.weight
model.embeddings.LayerNorm.bias
model.encoder.layer.0.attention.self.query.weight
model.encoder.layer.0.attention.self.query.bias
model.encoder.layer.0.attention.self.key.weight
model.encoder.layer.0.attention.self.key.bias
model.encoder.layer.0.attention.self.value.weight
model.encoder.layer.0.attention.self.value.bias
model.encoder.layer.0.attention.output.dense.weight
model.encoder.layer.0.attention.output.dense.bias
model.encoder.layer.0.attention.output.LayerNorm.weight
model.encoder.layer.0.attention.output.LayerNorm.bias
model.encoder.layer.0.intermediate.dense.weight
model.encoder.layer.0.intermediate.dense.bias
model.encoder.layer.0.output.dense.weight
model.encoder.layer.0.output.dense.bias
model.encoder.layer.0.output.LayerNorm.weight
model.encoder.layer.0.output.LayerNorm.bias
model.encoder.layer.1.attention.self

In [2]:
len(texts)

100000